### Feature Extraction
* **Products used:** 
[dem_cop_30](https://explorer.digitalearth.africa/products/s2_l2a), [s2_l2a](https://explorer.digitalearth.africa/products/dem_cop_90), [dem_srtm](https://explorer.digitalearth.africa/products/dem_srtm), [dem_srtm_deriv](https://explorer.digitalearth.africa/products/dem_srtm_deriv)

## Background:

Training data extraction plays a crucial role in training machine learning models. The process involves extracting relevant feature layers from a geospatial dataset based on predefined geometries or regions of interest. This enables the creation of accurate and reliable classification models for various applications such as land cover mapping, crop monitoring, and environmental analysis.

To facilitate this task, the open-data-cube provides a powerful function called "collect_training_data." This function is part of the deafrica_tools.classification script and is specifically designed to extract training data from the open-data-cube using geometries defined within a GeoJSON file. The GeoJSON file contains the spatial boundaries or polygons that delineate the regions of interest for which training data needs to be extracted.

## Description:

This notebook focuses on the extraction of training data (feature layers) from the open-data-cube using geometries defined within a GeoJSON file. It follows a step-by-step approach to guide users in utilizing the "collect_training_data" function effectively. The goal is to enable users to extract the appropriate training data for their specific use case.

The main steps in this notebook are as follows:

1. **Previewing the Training Data:** The notebook starts by plotting the polygons from the training data on a basemap. This visualization provides users with a visual representation of the regions of interest for which training data will be extracted.

2. **Defining the Feature Layer Function:** Next, a feature layer function is defined. This function specifies the set of feature layers to be extracted from the open-data-cube. These layers are carefully selected based on their relevance to the classification task at hand.

3. **Extracting Training Data:** The "collect_training_data" function is then employed to extract the training data from the datacube. It utilizes the predefined geometries from the GeoJSON file and retrieves the corresponding feature layers. This step ensures that the extracted data aligns precisely with the defined regions of interest.

4. **Exporting Training Data:** Finally, the extracted training data is exported and saved to disk. This facilitates its subsequent use in other scripts or machine learning workflows for training classification models.

By following the steps outlined in this notebook, users can leverage the "collect_training_data" function to efficiently extract training data from the open-data-cube. 

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [ ]:
%matplotlib inline
# import io
import os 
# import math
# import datacube
# import warnings
# import rioxarray
# import rasterio
#import numpy as np
import pandas as pd
import xarray as xr
import xarray as xr
import geopandas as gpd
#import matplotlib.pyplot as plt
from pathlib import Path
import rioxarray as rx
import pandas as pd
import xarray as xr

# from datacube.testutils.io import rio_slurp_xarray
# from deafrica_tools.datahandling import load_ard
# from deafrica_tools.plotting import map_shapefile
# from deafrica_tools.bandindices import calculate_indices
#from classification import collect_training_data


## Analysis parameters
 * path: The path to the input vector file from which we will extract training data. A default geojson is provided.
 * field: This is the name of column in your shapefile attribute table that contains the class labels. The class labels must be integers

In [ ]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
SKYKOMISH_HUC8_ID = "17110009"
prefix = SKYKOMISH_HUC8_ID
field = "class_id"
path = f"data/{prefix}_training_samples.geojson"
# swap out to wherever upstream processes are saving the rasters
ras_dir = "C:/Users/Arianna/Documents/rf_testing/rasters"

# Load input data shapefile
training_points = gpd.read_file(path)

In [ ]:
# # Set a flag to convert to polygons:
# use_polygons = False

# if use_polygons:
#     # Convert from lat,lon to EPSG:6933 (projection in metres)
#     training_points = training_points.to_crs("EPSG:6933")

#     # Buffer geometry to get a square - only if trying to sample multiple pixels
#     buffer_radius_m = 10
#     training_points.geometry = training_points.geometry.buffer(
#         buffer_radius_m, cap_style=3
#     )

#### Plot on interactive map 

In [ ]:
# points = training_points
# points.explore(
#     tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
#     attr="Imagery @2022 Landsat/Copernicus, Map data @2022 Google",
#     popup=True,
#     cmap="viridis",
#     style_kwds=dict(radius=5, color="red", fillOpacity=0.8, fillColor="red", weight=3),
# )

In [ ]:
# dem_file = "../../../data/processed/17110009_Skykomish_HE_DEM_3m.tif"
# CRS = rx.open_rasterio(ras_paths[0], chunks=True).squeeze(drop=True).rio.crs

In [ ]:
# note: the raster outputs of the TWI scripts and Dan's makegrids ended up
# with origins offset 0.5m from each other in the x and y directions.
# when this is true, load_training_data will yield errors and the sampling
# in the next cell will get messed up WITHOUT showing any errors
def load_training_data():
    return xr.merge(
        [
            rx.open_rasterio(f, chunks=True).squeeze(drop=True).to_dataset(name=f.stem)
            for f in Path(ras_dir).glob("*.tif")
        ]
    )

training_xr = load_training_data()


In [ ]:
def sample_xr(xr_ds: xr.Dataset, points: gpd.GeoDataFrame):
    points_proj = training_points
    pts_da = points_proj.assign(
        x=points_proj.geometry.x, y=points_proj.geometry.y
    ).to_xarray()

    # a dataframe or series (for a single point)
    pt_values_i = (
        xr_ds.sel(pts_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
    )

    if isinstance(pt_values_i, pd.Series):
        pt_values_i = pt_values_i.to_frame().transpose()
        pt_values_i.index = points.index

    return pd.concat([points_proj, pt_values_i], axis=1)

pd_training_features = sample_xr(training_xr, training_points)
pd_training_features

### Export training features

In [ ]:
# set the name and location of the output file
# output_file = "results/training_features.txt"
output_file = f"data/{prefix}_training_features.csv"
# Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=" ")

In [ ]:
# create geopandas dataframe
gpd_training_features = gpd.GeoDataFrame(
    pd_training_features,
    geometry="geometry",
)

#####  Add a column for binary (wetland/non-wetland) classification
This block ensures that both binary and multi-class classification labels are properly set up from the original `class_id` field:

- If `class_id` contains only two values (0 and 1), it is assumed to be binary, and is renamed to `class_id_binary`.
- If `class_id` includes additional wetland types, a new binary column `class_id_binary` is created:
  - `1` for any wetland type (i.e., values not equal to 0)
  - `0` for non-wetland (i.e., value equal to 0)
  - The original column is then renamed to `class_id_type` for use in multi-class classification.

In [ ]:
# Check if unique values in 'class_id' are only 0 and 1
unique_values = gpd_training_features["class_id"].unique()
if len(unique_values) == 2 and set(unique_values) == {0, 1}:
    # Replace 'class_id' with 'class_id_binary'
    gpd_training_features.rename(columns={"class_id": "class_id_binary"}, inplace=True)
else:
    # Create 'class_id_binary' column based on condition
    gpd_training_features["class_id_binary"] = gpd_training_features["class_id"].apply(
        lambda x: 1 if x != 0 else 0
    )
    gpd_training_features.rename(columns={"class_id": "class_id_type"}, inplace=True)

# Insert the new column at the second position
gpd_training_features.insert(
    0, "class_id_binary", gpd_training_features.pop("class_id_binary")
)
print(gpd_training_features.columns)

In [ ]:
# Replace non-zero values in the 'class_id' column with 1
gpd_training_features["class_id_binary"] = gpd_training_features["class_id_type"].apply(
    lambda x: 1 if x != 0 else 0
)
# Insert the new column at the second position
gpd_training_features.insert(
    1, "class_id_binary", gpd_training_features.pop("class_id_binary")
)
gpd_training_features

In [ ]:
# save as geojson file
results_dir = Path("results")
os.makedirs(results_dir, exist_ok=True)
geojson_file = f"results/{prefix}_training_features.geojson"
gpd_training_features.to_file(geojson_file, driver="GeoJSON")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")